<a href="https://colab.research.google.com/github/Deep-Dhaduk/AutoGluon/blob/main/Fraud-detection/fraud_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install --quiet autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 20.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.9/454.9 kB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.3/487.3 kB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.7/189.7 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.0/71.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 13.2 MB/s eta 0:0

In [3]:
import pandas as pd
import numpy as np
from autogluon.tabular import TabularPredictor

directory = '~/IEEEfraud/'  # directory where you have downloaded the data CSV files from the competition
label = 'isFraud'  # name of target variable to predict in this competition
eval_metric = 'roc_auc'  # Optional: specify that competition evaluation metric is AUC
save_path = directory + 'AutoGluonModels/'  # where to store trained models

train_identity = pd.read_csv('/content/drive/MyDrive/AutoGluon-Datasets/ieee-fraud-detection/train_identity.csv')
train_transaction = pd.read_csv('/content/drive/MyDrive/AutoGluon-Datasets/ieee-fraud-detection/train_transaction.csv')

In [4]:
train_data = pd.merge(train_transaction, train_identity, on='TransactionID', how='left')

In [6]:
predictor = TabularPredictor(label=label, eval_metric=eval_metric, path=save_path, verbosity=3).fit(
    train_data, presets='medium', time_limit=360
)

results = predictor.fit_summary()

Preset alias specified: 'medium' maps to 'medium_quality'.
Verbosity: 3 (Detailed Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.12.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu Oct  2 10:42:05 UTC 2025
CPU Count:          8
GPU Count:          1
Memory Avail:       36.76 GB / 50.99 GB (72.1%)
Disk Space Avail:   184.95 GB / 235.68 GB (78.5%)
Presets specified: ['medium']
============ fit kwarg info ============
User Specified kwargs:
{'auto_stack': False}
Full kwargs:
{'_experimental_dynamic_hyperparameters': False,
 '_feature_generator_kwargs': None,
 '_save_bag_folds': None,
 'ag_args': None,
 'ag_args_ensemble': None,
 'ag_args_fit': None,
 'auto_stack': False,
 'calibrate': 'auto',
 'delay_bag_sets': False,
 'ds_args': {'clean_up_fits': True,
             'detection_time_frac': 0.25,
             'enable_callbacks': False,
             'enable_ray_logging': True,
             'ho

[50]	valid_set's binary_logloss: 0.100743
[100]	valid_set's binary_logloss: 0.0924349
[150]	valid_set's binary_logloss: 0.0884887
[200]	valid_set's binary_logloss: 0.0856469
[250]	valid_set's binary_logloss: 0.0834745
[300]	valid_set's binary_logloss: 0.0818649
[350]	valid_set's binary_logloss: 0.0805544
[400]	valid_set's binary_logloss: 0.079232
[450]	valid_set's binary_logloss: 0.0784206
[500]	valid_set's binary_logloss: 0.0768095
[550]	valid_set's binary_logloss: 0.0759847
[600]	valid_set's binary_logloss: 0.075066
[650]	valid_set's binary_logloss: 0.0743114
[700]	valid_set's binary_logloss: 0.0736169
[750]	valid_set's binary_logloss: 0.0727746
[800]	valid_set's binary_logloss: 0.072003
[850]	valid_set's binary_logloss: 0.0714918
[900]	valid_set's binary_logloss: 0.0708359
[950]	valid_set's binary_logloss: 0.0705871
[1000]	valid_set's binary_logloss: 0.0700511
[1050]	valid_set's binary_logloss: 0.0695724
[1100]	valid_set's binary_logloss: 0.0685792
[1150]	valid_set's binary_logloss:

	Ran out of time, early stopping on iteration 5417. Best iteration is:
	[5410]	valid_set's binary_logloss: 0.0507601
Saving /root/IEEEfraud/AutoGluonModels/models/LightGBMXT/model.pkl
Saving /root/IEEEfraud/AutoGluonModels/utils/attr/LightGBMXT/y_pred_proba_val.pkl
	0.968	 = Validation score   (roc_auc)
	327.09s	 = Training   runtime
	1.01s	 = Validation runtime
	5822.6	 = Inference  throughput (rows/s | 5906 batch size)
Saving /root/IEEEfraud/AutoGluonModels/models/trainer.pkl
Skipping LightGBM due to lack of time remaining.
Saving /root/IEEEfraud/AutoGluonModels/models/trainer.pkl
Skipping RandomForestGini due to lack of time remaining.
Saving /root/IEEEfraud/AutoGluonModels/models/trainer.pkl
Skipping RandomForestEntr due to lack of time remaining.
Saving /root/IEEEfraud/AutoGluonModels/models/trainer.pkl
Skipping CatBoost due to lack of time remaining.
Saving /root/IEEEfraud/AutoGluonModels/models/trainer.pkl
Skipping ExtraTreesGini due to lack of time remaining.
Saving /root/IEEEf

*** Summary of fit() ***
Estimated performance of each model:
                 model  score_val eval_metric  pred_time_val    fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0           LightGBMXT    0.96797     roc_auc       1.014324  327.086754                1.014324         327.086754            1       True          1
1  WeightedEnsemble_L2    0.96797     roc_auc       1.016265  327.092953                0.001940           0.006200            2       True          2
Number of models trained: 2
Types of models trained:
{'LGBModel', 'WeightedEnsembleModel'}
Bagging used: False 
Multi-layer stack-ensembling used: False 
Feature Metadata (Processed):
(raw dtype, special dtypes):
('category', []) :  31 | ['ProductCD', 'card4', 'card6', 'P_emaildomain', 'R_emaildomain', ...]
('float', [])    : 395 | ['TransactionAmt', 'card2', 'card3', 'card5', 'addr1', ...]
('int', [])      :   3 | ['TransactionID', 'TransactionDT', 'card1']
Plot summary of models

In [27]:
predictor.leaderboard(silent=True)

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBMXT,0.96797,roc_auc,1.014324,327.086754,1.014324,327.086754,1,True,1
1,WeightedEnsemble_L2,0.96797,roc_auc,1.016265,327.092953,0.001940,0.006200,2,True,2


In [22]:
test_identity = pd.read_csv('/content/drive/MyDrive/AutoGluon-Datasets/ieee-fraud-detection/test_identity.csv')
test_transaction = pd.read_csv('/content/drive/MyDrive/AutoGluon-Datasets/ieee-fraud-detection/test_transaction.csv')
test_data = pd.merge(test_transaction, test_identity, on='TransactionID', how='left')  # same join applied to training files

# Rename columns with hyphens to underscores to match the training data
test_data.columns = test_data.columns.str.replace('-', '_')

In [16]:
test_identity.head()

,TransactionID,id-01,id-02,id-03,id-04,id-05,id-06,id-07,id-08,id-09,...,id-31,id-32,id-33,id-34,id-35,id-36,id-37,id-38,DeviceType,DeviceInfo
0,3663586,-45.0,280290.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,...,chrome 67.0 for android,NaN,NaN,NaN,F,F,T,F,mobile,MYA-L13 Build/HUAWEIMYA-L13
1,3663588,0.0,3579.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,...,chrome 67.0 for android,24.0,1280x720,match_status:2,T,F,T,T,mobile,LGLS676 Build/MXB48T
2,3663597,-5.0,185210.0,NaN,NaN,1.0,0.0,NaN,NaN,NaN,...,ie 11.0 for tablet,NaN,NaN,NaN,F,T,T,F,desktop,Trident/7.0
3,3663601,-45.0,252944.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,...,chrome 67.0 for android,NaN,NaN,NaN,F,F,T,F,mobile,MYA-L13 Build/HUAWEIMYA-L13
4,3663602,-95.0,328680.0,NaN,NaN,7.0,-33.0,NaN,NaN,NaN,...,chrome 67.0 for android,NaN,NaN,NaN,F,F,T,F,mobile,SM-G9650 Build/R16NW


In [17]:
test_transaction.head()

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,3663549,18403224,31.95,W,10409,111.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3663550,18403263,49.00,W,4272,111.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3663551,18403310,171.00,W,4476,574.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3663552,18403310,284.95,W,10989,360.0,150.0,visa,166.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3663553,18403317,67.95,W,18018,452.0,150.0,mastercard,117.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
print(test_data.columns)

Index(['TransactionID', 'TransactionDT', 'TransactionAmt', 'ProductCD',
       'card1', 'card2', 'card3', 'card4', 'card5', 'card6',
       ...
       'id-31', 'id-32', 'id-33', 'id-34', 'id-35', 'id-36', 'id-37', 'id-38',
       'DeviceType', 'DeviceInfo'],
      dtype='object', length=433)


In [ ]:
predictor.positive_class

In [ ]:
predictor.class_labels  # classes in this list correspond to columns of predict_proba() output

In [ ]:
y_predproba = predictor.predict_proba(test_data, as_multiclass=False)

In [ ]:
submission = pd.read_csv(directory+'sample_submission.csv')
submission['isFraud'] = y_predproba
submission.head()
submission.to_csv(directory+'my_submission.csv', index=False)